In [1]:
import keras
import pandas as pd
from keras import layers
import tensorflow as tf

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
import numpy as np
import os

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error as mse
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.layers import Dense, InputLayer, LSTM, Dropout, Bidirectional

# from keras import ops
import numpy as np

In [3]:
DC_POWER_INDEX = 0
LEARNING_RATE = 0.001
RANDOM_STATE = 44
START_TIME = '2020-06-07 00:30:00'
END_TIME = '2020-06-07 04:00:00'
BATCH_SIZE = 1024

def fix_randomness():
    tf.random.set_seed(RANDOM_STATE)

    from numpy.random import seed
    seed(RANDOM_STATE)
    keras.utils.set_random_seed(RANDOM_STATE)

fix_randomness()

MULTIVARIATE

In [4]:
EPOCHS = 500
MULTIV_TRAINING_WINDOW = 40
lead_time = 20

In [5]:
# Creating the model input for multivariate forecasting
def df_to_model_input2(df_np, col_index, window_size):
    df_np = df_np.to_numpy()
    x = []
    y = []

    for i in range(len(df_np) - window_size - lead_time):
        row = [r for r in df_np[i:i+window_size]]
        x.append(row)

        label = df_np[i+lead_time+window_size][col_index]
        y.append(label)

    return np.array(x), np.array(y)

In [6]:
multiv_teacher_path = 'models/girasol/multiv_teacher_model.keras'
multiv_student_path = 'models/girasol/multiv_student_model.keras'

In [7]:
train = pd.read_csv('/.../girasol_met/train.csv')
test = pd.read_csv('/.../girasol_met/val.csv')

In [8]:
NUMBER_OF_FEATURES = len(train.columns)

In [9]:
scaler = MinMaxScaler()
train = pd.DataFrame(scaler.fit_transform(train), columns=train.columns)
test = pd.DataFrame(scaler.transform(test), columns=test.columns)

In [10]:
def get_train_test_val(x, y):
    return x[:train_len], y[:train_len], x[train_len:], y[train_len:]

x, y = df_to_model_input2(train, 7, MULTIV_TRAINING_WINDOW)

ds_len = len(y)
train_len = int(0.8*ds_len)

x_train, y_train, x_val, y_val = get_train_test_val(x, y)
x_test, y_test = df_to_model_input2(test, 7, MULTIV_TRAINING_WINDOW)

In [11]:
x_test.shape,  y_test.shape

((25163, 40, 8), (25163,))

In [12]:
teacher_hidden_layer_units, teacher_dense_params = 512, 8
student_hidden_layer_units, student_dense_params = 16, 8

In [13]:
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def calculate_mape(scaler, y_pred_scaled, target_column_index=-1):

    # Create dummy arrays to match the scaler's expected input shape
    dummy_data_pred = np.zeros((y_pred_scaled.shape[0], NUMBER_OF_FEATURES))
    dummy_data_test = np.zeros((y_test.shape[0], NUMBER_OF_FEATURES))

    # Insert the predicted values and y_test into the correct column
    dummy_data_pred[:, target_column_index] = y_pred_scaled[:, 0]
    dummy_data_test[:, target_column_index] = y_test[:] # it kept that way to be able to modify if needed

    # Perform inverse transform to get the unscaled predictions and y_test
    unscaled_predictions = scaler.inverse_transform(dummy_data_pred)[:, target_column_index]
    y_test_unscaled = scaler.inverse_transform(dummy_data_test)[:, target_column_index]



    mape = mean_absolute_percentage_error(y_test_unscaled, unscaled_predictions)*100
    return mape

def calculate_metrics(y_true, y_pred):
    """
    Calculate NMSE, NRMSE, and NMAE normalized by the average of true values.
    
    Parameters:
        y_true (array-like): True values.
        y_pred (array-like): Predicted values.
        
    Returns:
        dict: A dictionary containing NMSE, NRMSE, and NMAE.
    """
    # Convert inputs to numpy arrays for consistency
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Mean Squared Error (MSE) and Mean Absolute Error (MAE) using sklearn
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    mape = calculate_mape(scaler=scaler, y_pred_scaled=y_pred)
    # Variance and mean of true values
    avg_true = np.mean(y_true)
    
    # Calculate metrics
    nrmse = np.sqrt(mse) / avg_true
    nmae = mae / avg_true

    print(f"NRMSE (Normalized Root Mean Squared Error): {nrmse:.4f}")
    print(f"NMAE (Normalized Mean Absolute Error): {nmae:.4f}")
    print(f"MAPE (Mean Absolute Percentage Error): {mape:.4f}") 
    
    # Return metrics as a dictionary
    return nrmse, nmae, mape

In [14]:
from sklearn.preprocessing import MinMaxScaler

RANDOM_STATE = 44
tf.random.set_seed(RANDOM_STATE)
from numpy.random import seed
seed(RANDOM_STATE)
keras.utils.set_random_seed(RANDOM_STATE)

def run_teacher_multiv():

    # Create the model
    model = Sequential()
    model.add(InputLayer((MULTIV_TRAINING_WINDOW, NUMBER_OF_FEATURES)))
    model.add(LSTM(teacher_hidden_layer_units))
    model.add(Dropout(0.5))
    model.add(Dense(teacher_dense_params, 'relu'))
    model.add(Dense(1, 'linear'))

    # Model checkpoint
    model_file = multiv_teacher_path

    cp = ModelCheckpoint(model_file, save_best_only=True)

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=5,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True  # Restore the weights of the best epoch after stopping
    )

    # Compile the model
    model.compile(
        loss=MeanSquaredError(),
        optimizer=Adam(learning_rate=LEARNING_RATE),
        metrics=[MeanSquaredError()]
    )
   
    # Train the model
    model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[cp, early_stopping]
    )

run_teacher_multiv()

teacher = load_model(multiv_teacher_path)
print("Teacher Validation RMSE") 
teacher.evaluate(x_val, y_val)

Epoch 1/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 24s 638ms/step - loss: 0.1013 - mean_squared_error: 0.1012 - val_loss: 0.0064 - val_mean_squared_error: 0.0078
Epoch 2/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 25s 685ms/step - loss: 0.0094 - mean_squared_error: 0.0094 - val_loss: 0.0046 - val_mean_squared_error: 0.0051
Epoch 3/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 27s 721ms/step - loss: 0.0077 - mean_squared_error: 0.0077 - val_loss: 0.0045 - val_mean_squared_error: 0.0050
Epoch 4/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 706ms/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0047 - val_mean_squared_error: 0.0050
Epoch 5/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 693ms/step - loss: 0.0071 - mean_squared_error: 0.0071 - val_loss: 0.0046 - val_mean_squared_error: 0.0050
Epoch 6/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 701ms/step - loss: 0.0070 - mean_squared_error: 0.0070 - val_loss: 0.0045 - val_mean_squared_error: 0.0049
Epoch 7/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 701ms/step - loss: 0.0068 - mean_squared_error: 0.0068 - 

[0.004304586444050074, 0.004317599814385176]

In [15]:
teacher = load_model(multiv_teacher_path)

# Make predictions on the scaled test set
y_pred = teacher.predict(x_test)

nrmse, nmae, mape = calculate_metrics(y_test, y_pred)
# Store the results in a dictionary for the teacher
teacher_results = {
    "NRMSE": nrmse,
    "NMAE": nmae,
    "MAPE": mape
}

# Print the results in a single row
print(f"Teacher Results: NRMSE = {teacher_results['NRMSE']:.4f}, NMAE = {teacher_results['NMAE']:.4f}, MAPE = {teacher_results['MAPE']:.4f}")


787/787 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step
NRMSE (Normalized Root Mean Squared Error): 0.1703
NMAE (Normalized Mean Absolute Error): 0.1163
MAPE (Mean Absolute Percentage Error): 16.4193
Teacher Results: NRMSE = 0.1703, NMAE = 0.1163, MAPE = 16.4193


In [16]:
from sklearn.preprocessing import MinMaxScaler

RANDOM_STATE = 44
tf.random.set_seed(RANDOM_STATE)
from numpy.random import seed
seed(RANDOM_STATE)
keras.utils.set_random_seed(RANDOM_STATE)

def run_student_multiv():

    # Create the model
    model = Sequential()
    model.add(InputLayer((MULTIV_TRAINING_WINDOW, NUMBER_OF_FEATURES)))
    model.add(LSTM(student_hidden_layer_units))
    model.add(Dropout(0.5))
    model.add(Dense(student_dense_params, 'relu'))
    model.add(Dense(1, 'linear'))

    # Model checkpoint
    model_file = multiv_student_path

    cp = ModelCheckpoint(model_file, save_best_only=True)

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=5,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True  # Restore the weights of the best epoch after stopping
    )

    # Compile the model
    model.compile(
        loss=MeanSquaredError(),
        optimizer=Adam(learning_rate=LEARNING_RATE),
        metrics=[MeanSquaredError()]
    )
   
    # Train the model
    model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=500,
        callbacks=[cp, early_stopping]
    )

run_student_multiv()

student = load_model(multiv_student_path)
print("Student Validation RMSE") 
student.evaluate(x_val, y_val)

Epoch 1/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.1986 - mean_squared_error: 0.1985 - val_loss: 0.0150 - val_mean_squared_error: 0.0165
Epoch 2/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0463 - mean_squared_error: 0.0463 - val_loss: 0.0174 - val_mean_squared_error: 0.0176
Epoch 3/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0327 - mean_squared_error: 0.0327 - val_loss: 0.0124 - val_mean_squared_error: 0.0123
Epoch 4/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0263 - mean_squared_error: 0.0263 - val_loss: 0.0119 - val_mean_squared_error: 0.0114
Epoch 5/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0220 - mean_squared_error: 0.0220 - val_loss: 0.0106 - val_mean_squared_error: 0.0102
Epoch 6/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0193 - mean_squared_error: 0.0193 - val_loss: 0.0085 - val_mean_squared_error: 0.0084
Epoch 7/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0171 - mean_squared_error: 0.0171 - val_loss: 0.00

[0.005547190085053444, 0.00555793521925807]

In [17]:
student = load_model(multiv_student_path)

# Make predictions on the scaled test set
y_pred = student.predict(x_test)

nrmse, nmae, mape = calculate_metrics(y_test, y_pred)

# Store the results in a dictionary
student_results = {
    "NRMSE": nrmse,
    "NMAE": nmae,
    "MAPE": mape
}
# Print the results in a single row
print(f"Student Results: NRMSE = {student_results['NRMSE']:.4f}, NMAE = {student_results['NMAE']:.4f}, MAPE = {student_results['MAPE']:.4f}")


787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 827us/step
NRMSE (Normalized Root Mean Squared Error): 0.1944
NMAE (Normalized Mean Absolute Error): 0.1472
MAPE (Mean Absolute Percentage Error): 23.3606
Student Results: NRMSE = 0.1944, NMAE = 0.1472, MAPE = 23.3606


In [18]:
from tensorflow.keras.saving import register_keras_serializable

@register_keras_serializable()
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student
        self._loss_tracker = keras.metrics.Mean(name="loss")

    def compile(
        self,
        optimizer,
        loss,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        loss_threshold,
        alpha,
        temperature

    ):
        """Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super().compile(loss=loss, optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature
        self.loss_threshold = loss_threshold

    def get_config(self):
        config = {
            'student': self.student.to_json(),
            'teacher': self.teacher.to_json()
        }

        return config

    @classmethod
    def from_config(cls, config):
        student = tf.keras.models.model_from_json(config.pop('student'))
        teacher = tf.keras.models.model_from_json(config.pop('teacher'))
        return cls(student=student, teacher=teacher, **config)

    def train_step(self, data):
        # Unpack the data
        x, y = data

        with tf.GradientTape() as tape:
            # Forward pass
            y_pred = self.student(x, training=True)
            # Compute the loss value
            loss = self.compute_loss(x, y, y_pred)

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`
        self.compiled_metrics.update_state(y, y_pred)

        # Return a dict mapping metric names to current value
        results =  {m.name: m.result() for m in self.metrics}

        results['total loss (train)'] = loss if isinstance(loss, float) else tf.reduce_mean(loss)

        return results


    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Forward pass to get student's predictions
        y_pred = self.student(x, training=False)

        # Compute the validation loss
        val_loss = self.compute_loss(x, y, y_pred)

        # Manually update the metrics for validation
        self.compiled_metrics.update_state(y, y_pred)

        # Collect results for all metrics
        results = {m.name: m.result() for m in self.metrics}

        # Ensure 'val_loss' is properly reduced to a scalar and reported
        results['total loss (val)'] = val_loss if isinstance(val_loss, float) else tf.reduce_mean(val_loss)

        return results



    def compute_loss(
        # self, x=None, y=None, y_pred=None, sample_weight=None, allow_empty=False
        self, x=None, y=None, y_pred=None, sample_weight=None, allow_empty=False

    ):

        mse = MeanSquaredError()

        # Compute predictions by the teacher model
        teacher_pred = self.teacher(x, training=False)

        # Compute the MSE loss between true labels and student predictions
        student_loss = mse(y, y_pred)

        # Teacher loss is the
        temp = self.temperature
        # loss = self.alpha * student_loss + (1 - self.alpha) * teacher_loss
        loss = self.alpha * student_loss + (1 - self.alpha) * (mse(y_pred/temp, teacher_pred/temp)* (temp ** 2))


        return loss

    def call(self, x):
        return self.student(x)

In [19]:
def run_multiv_distillation(teacher, alpha, threshold, temperature, file_name, window_size):

    # Create the model
    model = Sequential()
    model.add(InputLayer((MULTIV_TRAINING_WINDOW, NUMBER_OF_FEATURES)))
    model.add(LSTM(student_hidden_layer_units))
    model.add(Dropout(0.5))
    model.add(Dense(student_dense_params, 'relu'))
    model.add(Dense(1, 'linear'))

    # Create the distiller class
    distiller = Distiller(student=model, teacher=teacher)

    # Compile the distiller class
    distiller.compile(
    optimizer = Adam(learning_rate=LEARNING_RATE),
    loss = MeanSquaredError(),
    metrics = [RootMeanSquaredError()],
    student_loss_fn = MeanSquaredError(),
    distillation_loss_fn = distiller.compute_loss, #MeanSquaredError(),
    loss_threshold = threshold,
    alpha=alpha,
    temperature=temperature
    )

    dummy_x = tf.random.normal([1, *((window_size, NUMBER_OF_FEATURES))])  # Replace `input_shape` with the actual shape of your input
    _ = distiller(dummy_x) 

    d_check = ModelCheckpoint(file_name, monitor='root_mean_squared_error',save_best_only=True)

    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=5,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True  # Restore the weights of the best epoch after stopping
    )

    distiller.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=EPOCHS, callbacks=[d_check, early_stopping])

    distilled_student = load_model(file_name)
    # Make predictions using the distilled student model
    y_pred = distilled_student.predict(x_val)

    # Calculate the root mean squared error (RMSE)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print("Distilled Student RMSE:", rmse)

In [20]:
alphas = [0.3, 0.5, 0.7]
temps = [5]
threshold = 0

# Initialize a 2D list to store RMSE values
rmse_matrix = []

# Iterate over alphas and temps
for alpha in alphas:
    row_rmse = []
    for temp in temps:

        RANDOM_STATE = 44
        tf.random.set_seed(RANDOM_STATE)
        from numpy.random import seed
        seed(RANDOM_STATE)
        keras.utils.set_random_seed(RANDOM_STATE)

        multiv_distill_file_name = 'models/girasol/distils/multiv_distil_'+str(alpha)+'_'+str(temp)+'/multiv_distil.keras'
        
        run_multiv_distillation(teacher, alpha, threshold, temp,  multiv_distill_file_name, MULTIV_TRAINING_WINDOW)
        
        # Load the model
        multiv_distill_file_name = f'models/girasol/distils/multiv_distil_{alpha}_{temp}/multiv_distil.keras'
        student_kd = load_model(multiv_distill_file_name)

        # Make predictions on the scaled test set
        y_pred = student_kd.predict(x_test)

        nrmse, nmae, mape = calculate_metrics(y_test, y_pred)
        
        row_rmse.append(nrmse)

    # Append the row of RMSE values to the matrix
    rmse_matrix.append(row_rmse)

# Convert the list to a NumPy array for plotting
rmse_matrix = np.array(rmse_matrix)

Epoch 1/500


/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/backend/tensorflow/trainer.py:639: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


1162/1162 ━━━━━━━━━━━━━━━━━━━━ 61s 51ms/step - root_mean_squared_error: 0.1855 - loss: 0.4852 - total loss (train): 0.0144 - val_loss: 0.5406 - val_total loss (val): 0.0059
Epoch 2/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 61s 53ms/step - root_mean_squared_error: 0.0923 - loss: 0.5287 - total loss (train): 0.0042 - val_loss: 0.5535 - val_total loss (val): 0.0031
Epoch 3/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 61s 52ms/step - root_mean_squared_error: 0.0860 - loss: 0.5292 - total loss (train): 0.0033 - val_loss: 0.5350 - val_total loss (val): 0.0031
Epoch 4/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 68s 58ms/step - root_mean_squared_error: 0.0825 - loss: 0.5290 - total loss (train): 0.0028 - val_loss: 0.5242 - val_total loss (val): 0.0025
Epoch 5/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 62s 53ms/step - root_mean_squared_error: 0.0811 - loss: 0.5291 - total loss (train): 0.0026 - val_loss: 0.5011 - val_total loss (val): 0.0031
Epoch 6/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 60s 52ms/step - root_mean_squared_error: 0.0

/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


291/291 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step
Distilled Student RMSE: 0.07079261194396891
133/787 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step

...

1162/1162 ━━━━━━━━━━━━━━━━━━━━ 46s 39ms/step - root_mean_squared_error: 0.1855 - loss: 0.4835 - total loss (train): 0.0167 - val_loss: 0.5382 - val_total loss (val): 0.0127
Epoch 2/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 46s 39ms/step - root_mean_squared_error: 0.0927 - loss: 0.5266 - total loss (train): 0.0066 - val_loss: 0.5425 - val_total loss (val): 0.0072
Epoch 3/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 45s 39ms/step - root_mean_squared_error: 0.0864 - loss: 0.5272 - total loss (train): 0.0056 - val_loss: 0.5276 - val_total loss (val): 0.0079
Epoch 4/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 43s 37ms/step - root_mean_squared_error: 0.0831 - loss: 0.5269 - total loss (train): 0.0052 - val_loss: 0.5172 - val_total loss (val): 0.0051
Epoch 5/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 46s 39ms/step - root_mean_squared_error: 0.0818 - loss: 0.5270 - total loss (train): 0.0050 - val_loss: 0.5054 - val_total loss (val): 0.0038
Epoch 6/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 46s 40ms/step - root_mean_squared_error: 0.0

/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


291/291 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Distilled Student RMSE: 0.07477565368032711
119/787 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step

/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 921us/step
NRMSE (Normalized Root Mean Squared Error): 0.1895
NMAE (Normalized Mean Absolute Error): 0.1408
MAPE (Mean Absolute Percentage Error): 22.1054


In [21]:
import numpy as np
from tensorflow.keras.models import load_model

# Example inputs
alphas = [0.3, 0.5, 0.7]
temps = [5]
threshold = 0

# Initialize a dictionary to store results
student_kd_results = []

# Iterate over alphas and temps
for alpha in alphas:
    for temp in temps:
        multiv_distill_file_name = f'models/girasol/distils/multiv_distil_{alpha}_{temp}/multiv_distil.keras'
        
        # Load the model
        student_kd = load_model(multiv_distill_file_name)

        # Make predictions on the scaled test set
        y_pred = student_kd.predict(x_test)

        # Calculate metrics
        nrmse, nmae, mape = calculate_metrics(y_test, y_pred)
        
        # Add the results to the dictionary
        student_kd_results.append({
            "alpha": alpha,
            "temp": temp,
            "NRMSE": nrmse,
            "NMAE": nmae,
            "MAPE": mape
        })

# Print the results
for result in student_kd_results:
    print(f"Alpha: {result['alpha']}, Temp: {result['temp']}, NRMSE: {result['NRMSE']:.4f}, "
          f"NMAE: {result['NMAE']:.4f}, MAPE: {result['MAPE']:.4f}")

117/787 ━━━━━━━━━━━━━━━━━━━━ 0s 869us/step

/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 929us/step
NRMSE (Normalized Root Mean Squared Error): 0.1844
NMAE (Normalized Mean Absolute Error): 0.1343
MAPE (Mean Absolute Percentage Error): 21.8915
119/787 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step

/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 928us/step
NRMSE (Normalized Root Mean Squared Error): 0.1859
NMAE (Normalized Mean Absolute Error): 0.1363
MAPE (Mean Absolute Percentage Error): 21.7649
109/787 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step

/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 936us/step
NRMSE (Normalized Root Mean Squared Error): 0.1895
NMAE (Normalized Mean Absolute Error): 0.1408
MAPE (Mean Absolute Percentage Error): 22.1054
Alpha: 0.3, Temp: 5, NRMSE: 0.1844, NMAE: 0.1343, MAPE: 21.8915
Alpha: 0.5, Temp: 5, NRMSE: 0.1859, NMAE: 0.1363, MAPE: 21.7649
Alpha: 0.7, Temp: 5, NRMSE: 0.1895, NMAE: 0.1408, MAPE: 22.1054


RESULTS

Efficiency Metrics

In [22]:
teacher = load_model(multiv_teacher_path)

# 1. Print model summary (Layer details and parameter count)
print("Model Summary:")
teacher.summary()

# 2. Get the total number of parameters
total_params = teacher.count_params()
print(f'\nTotal number of parameters: {total_params}')

# 3. Get the size of the model file on disk
model_size = os.path.getsize(multiv_teacher_path) / 1024  #a Convert from bytes to MB
print(f'Model size on disk: {model_size:.2f} KB')

Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 512)            │     1,067,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │         4,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,213,365 (12.26 MB)

 Trainable params: 1,071,121 (4.09 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,142,244 (8.17 MB)


Total number of parameters: 1071121
Model size on disk: 12582.91 KB


In [23]:
student = load_model(multiv_student_path)

# 1. Print model summary (Layer details and parameter count)
print("Model Summary:")
student.summary()

# 2. Get the total number of parameters
total_params = student.count_params()
print(f'\nTotal number of parameters: {total_params}')

# 3. Get the size of the model file on disk
model_size = os.path.getsize(multiv_student_path) / 1024  #a Convert from bytes to MB
print(f'Model size on disk: {model_size:.2f} KB')

Model Summary:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 16)             │         1,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,237 (20.46 KB)

 Trainable params: 1,745 (6.82 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,492 (13.64 KB)


Total number of parameters: 1745
Model size on disk: 51.17 KB


Error Metrics

In [24]:
# Print the results in a single row
print(f"Teacher Results: NRMSE = {teacher_results['NRMSE']:.4f}, NMAE = {teacher_results['NMAE']:.4f}, MAPE = {teacher_results['MAPE']:.4f}")

# Print the results in a single row
print(f"Student Results: NRMSE = {student_results['NRMSE']:.4f}, NMAE = {student_results['NMAE']:.4f}, MAPE = {student_results['MAPE']:.4f}")

print('Student KD Results')
# Print the results
for result in student_kd_results:
    print(f"Alpha: {result['alpha']}, Temp: {result['temp']}, NRMSE: {result['NRMSE']:.4f}, "
          f"NMAE: {result['NMAE']:.4f}, MAPE: {result['MAPE']:.4f}")

Teacher Results: NRMSE = 0.1703, NMAE = 0.1163, MAPE = 16.4193
Student Results: NRMSE = 0.1944, NMAE = 0.1472, MAPE = 23.3606
Student KD Results
Alpha: 0.3, Temp: 5, NRMSE: 0.1844, NMAE: 0.1343, MAPE: 21.8915
Alpha: 0.5, Temp: 5, NRMSE: 0.1859, NMAE: 0.1363, MAPE: 21.7649
Alpha: 0.7, Temp: 5, NRMSE: 0.1895, NMAE: 0.1408, MAPE: 22.1054


In [25]:
import seaborn as sns

alphas = [0.3, 0.5, 0.7]
temps = [5]

# Initialize a 2D list to store RMSE values
rmse_matrix = []

# Iterate over alphas and temps
for alpha in alphas:
    row_rmse = []
    for temp in temps:
        # Load the model
        multiv_distill_file_name = f'models/girasol/distils/multiv_distil_{alpha}_{temp}/multiv_distil.keras'
        student_kd = load_model(multiv_distill_file_name)

        # Make predictions on the scaled test set
        y_pred = student_kd.predict(x_test)

        nrmse, nmae, mape = calculate_metrics(y_test, y_pred)
        
        row_rmse.append(nrmse)

    # Append the row of RMSE values to the matrix
    rmse_matrix.append(row_rmse)

# Convert the list to a NumPy array for plotting
rmse_matrix = np.array(rmse_matrix)

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(rmse_matrix, annot=True, fmt='.2f', cmap='viridis', xticklabels=temps, yticklabels=alphas)
plt.xlabel('Temperature')
plt.ylabel('Alpha')
plt.title('Distilled Models on Multivariate Data')
plt.show()

ModuleNotFoundError: No module named 'seaborn'